In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
train = pd.read_csv('./output/train.csv')    
test = pd.read_csv('./output/test.csv')    
test2 = pd.read_csv('./output/test2.csv')
y_train=pd.read_csv('./output/y_train.csv')

In [3]:
test.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,x_dir,y_dir
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,42197,42617,0.0,0.0,0.0,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,-302.789015,-46798.074140
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,44497,44497,0.0,0.0,0.0,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07,-302.789015,-46798.074140
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,45267,47651,0.0,0.0,0.0,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07,0.000000,0.000000
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,48333,49393,0.0,0.0,0.0,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,7997.192219,102495.208073
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212,54632,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,NaN,NaN,-253.808439,-31208.509376


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

x_train = train.drop(columns=['hash','trajectory_id'])
x_test = test2.drop(columns=['hash','trajectory_id'])

model = GradientBoostingRegressor(n_estimators=100, 
                                  learning_rate=0.1,
                                  max_depth=10,
                                  min_samples_split=3 ,#2
                                  min_samples_leaf=2 ,#1 
                                  random_state=1234, 
                                  loss='ls',
                                 verbose=True)

model.fit(x_train, y_train.x_exit.values)
predx = model.predict(x_test)

      Iter       Train Loss   Remaining Time 
         1    66019715.6501           31.89m
         2    55265623.5280           31.29m
         3    46535331.4929           31.25m
         4    39460195.1292           30.96m
         5    33709820.4341           30.55m
         6    29032830.8908           30.12m
         7    25247765.1265           29.65m
         8    22167245.5894           29.39m
         9    19653023.4422           29.05m
        10    17608329.7050           28.64m


In [ ]:
model.fit(x_train, y_train.y_exit.values)
predy = model.predict(x_test)

In [ ]:

#pred = pd.DataFrame(data=pred,columns=['x_exit','y_exit'])
pred = pd.DataFrame({'x_exit':predx,'y_exit':predy})

In [ ]:
def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

X = pred['x_exit'].values 
Y = pred['y_exit'].values

city = [within_measure(x, y) for x,y in zip(X,Y)]

In [ ]:
submission= pd.DataFrame()
trajectory_id = test2['trajectory_id']
submission['id']=trajectory_id 
submission['target'] = city
submission.to_csv('./output/submission_gradientboosting.csv',index=False)    
submission[:20]

In [ ]:
min(city)

In [ ]:
max(city)